इस लेख में शामिल जप्टर नोटबुक [अध्याय 4 कोड बेस](https://github.com/datawhalechina/learn-nlp-with-transformers/tree/main/docs/%E7%AF%87%E7%) में है AB%A04-%E4%BD%BF%E7%94%A8Transformers%E8%A7%A3%E5%86%B3NLP%E4%BB%BB%E5%8A%A1).

प्रासंगिक डेटासेट और मॉडल को तुरंत डाउनलोड करने के लिए इस ट्यूटोरियल को सीधे Google कोलाब नोटबुक का उपयोग करके खोलने की अनुशंसा की जाती है।
यदि आप इस नोटबुक को Google कोलैब में खोल रहे हैं, तो आपको ट्रांसफॉर्मर और 🤗डेटासेट लाइब्रेरीज़ को इंस्टॉल करने की आवश्यकता हो सकती हैउन्हें स्थापित करने के लिए निम्नलिखित आदेश।

In [ ]:
! pip install datasets transformers "sacrebleu>=1.4.12,<2.0.0" sentencepiece

यदि आप इस नोटबुक को स्थानीय रूप से खोल रहे हैं, तो कृपया सुनिश्चित करें कि आपने ट्रांसफार्मर-क्विक-स्टार्ट-ज़ेड रीडमी फ़ाइल में सभी निर्भरताओं को ध्यान से पढ़ा और स्थापित किया है। आप इस नोटबुक का मल्टी-जीपीयू वितरित प्रशिक्षण संस्करण भी यहां पा सकते हैं। https://github.com/huggingface/transformers/tree/master/examples/seq2seq)।

# अनुवाद कार्य को हल करने के लिए ट्रांसफार्मर मॉडल को फाइन-ट्यून करें

इस नोटबुक में, हम दिखाएंगे कि प्राकृतिक भाषा प्रसंस्करण में अनुवाद कार्य को हल करने के लिए [🤗 ट्रांसफॉर्मर्स] (https://github.com/huggingface/transformers) रिपॉजिटरी से मॉडल का उपयोग कैसे करें। हम [WMT डेटासेट] का उपयोग करेंगे। (http://www.statmt.org/wmt16/) डेटासेट यह अनुवाद कार्यों के लिए सबसे अधिक उपयोग किए जाने वाले डेटासेट में से एक है।

एक उदाहरण नीचे दिया गया है:

![अनुवाद कार्य पर विजेट अनुमान](https://github.com/huggingface/notebooks/blob/master/examples/images/translation.png?raw=1)अनुवाद कार्य के लिए, हम दिखाएंगे कि एक सरल डेटासेट लोडिंग का उपयोग कैसे करें और ट्रांसफार्मर में ट्रेनर इंटरफ़ेस का उपयोग किए बिना संबंधित मॉडल के लिए मॉडल को कैसे ठीक करें।

In [2]:
model_checkpoint = "Helsinki-NLP/opus-mt-en-ro" 
# एक मॉडल चेकपॉइंट चुनें

जब तक पूर्व-प्रशिक्षित ट्रांसफार्मर मॉडल में एक seq2seq हेड परत होती है, यह नोटबुक सैद्धांतिक रूप से किसी भी अनुवाद कार्य को हल करने के लिए विभिन्न प्रकार के ट्रांसफार्मर मॉडल [मॉडल पैनल] (https://huggingface.co/models) का उपयोग कर सकता है।

इस लेख में, हम अनुवाद कार्यों के लिए पूर्व-प्रशिक्षित [`हेलसिंकी-एनएलपी/ओपस-एमटी-एन-रो`](https://huggingface.co/Helsinki-NLP/opus-mt-en-ro) चेकपॉइंट का उपयोग करते हैं .

## डेटा डाउनलोड करें

हम डेटा और संबंधित मेट्रिक्स को लोड करने के लिए 🤗 डेटासेट लाइब्रेरी का उपयोग करेंगे। डेटा लोडिंग और मीट्रिक लोडिंग के लिए केवल लोड_डेटासेट और लोड_मेट्रिक के उपयोग की आवश्यकता होती है। हम WMT डेटासेट में अंग्रेजी/रोमानियाई द्विभाषी अनुवाद का उपयोग करते हैं।

In [3]:
from datasets import load_dataset, load_metric

raw_datasets = load_dataset("wmt16", "ro-en")
metric = load_metric("sacrebleu")

Downloading: 2.81kB [00:00, 523kB/s]                    
Downloading: 3.19kB [00:00, 758kB/s]                    
Downloading: 41.0kB [00:00, 11.0MB/s]                   


Downloading: 100%|██████████| 225M/225M [00:18<00:00, 12.2MB/s]
Downloading: 100%|██████████| 23.5M/23.5M [00:16<00:00, 1.44MB/s]
Downloading: 100%|██████████| 38.7M/38.7M [00:03<00:00, 9.82MB/s]


Dataset wmt16 downloaded and prepared to /Users/niepig/.cache/huggingface/datasets/wmt16/ro-en/1.0.0/0d9fb3e814712c785176ad8cdb9f465fbe6479000ee6546725db30ad8a8b5f8a. Subsequent calls will reuse this data.


Downloading: 5.40kB [00:00, 2.08MB/s]                   


डेटासेट ऑब्जेक्ट स्वयं एक [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict) डेटा संरचना है, प्रशिक्षण सेट, सत्यापन सेट और परीक्षण सेट के लिए, आपको केवल इसकी आवश्यकता है संबंधित डेटा प्राप्त करने के लिए संबंधित कुंजी (ट्रेन, सत्यापन, परीक्षण) का उपयोग करना।

In [4]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 610320
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 1999
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 1999
    })
})

डेटा विभाजन कुंजी (ट्रेन, सत्यापन, या परीक्षण) और एक सबस्क्रिप्ट दिए जाने पर, आप डेटा देख सकते हैं।

In [5]:
raw_datasets["train"][0]
# हम देख सकते हैं कि एक अंग्रेजी वाक्य en एक रोमानियाई वाक्य ro से मेल खाता है

{'translation': {'en': 'Membership of Parliament: see Minutes',
  'ro': 'Componenţa Parlamentului: a se vedea procesul-verbal'}}

यह समझने के लिए कि डेटा कैसा दिखता है, निम्न फ़ंक्शन यादृच्छिक रूप से डेटासेट से कुछ उदाहरणों का चयन करेगा और उन्हें प्रदर्शित करेगा।

In [6]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [7]:
show_random_elements(raw_datasets["train"])

,translation
0,"{'en': 'I do not believe that this is the right course.', 'ro': 'Nu cred că acesta este varianta corectă.'}"
1,"{'en': 'A total of 104 new jobs were created at the European Chemicals Agency, which mainly supervises our REACH projects.', 'ro': 'Un total de 104 noi locuri de muncă au fost create la Agenția Europeană pentru Produse Chimice, care, în special, supraveghează proiectele noastre REACH.'}"
2,"{'en': 'In view of the above, will the Council say what stage discussions for Turkish participation in joint Frontex operations have reached?', 'ro': 'Care este stadiul negocierilor referitoare la participarea Turciei la operațiunile comune din cadrul Frontex?'}"
3,"{'en': 'We now fear that if the scope of this directive is expanded, the directive will suffer exactly the same fate as the last attempt at introducing 'Made in' origin marking - in other words, that it will once again be blocked by the Council.', 'ro': 'Acum ne temem că, dacă sfera de aplicare a directivei va fi extinsă, aceasta va avea exact aceeaşi soartă ca ultima încercare de introducere a marcajului de origine ""Made in”, cu alte cuvinte, că va fi din nou blocată la Consiliu.'}"
4,"{'en': 'The country dropped nine slots to 85th, with a score of 6.58.', 'ro': 'Ţara a coborât nouă poziţii, pe locul 85, cu un scor de 6,58.'}"


मेट्रिक [`datasets.Metric`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Metric) वर्ग का एक उदाहरण है। मीट्रिक और उपयोग के उदाहरण देखें:

In [8]:
metric

Metric(name: "sacrebleu", features: {'predictions': Value(dtype='string', id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id='references')}, usage: """
Produces BLEU scores along with its sufficient statistics
from a source against one or more references.

Args:
    predictions: The system stream (a sequence of segments)
    references: A list of one or more reference streams (each a sequence of segments)
    smooth: The smoothing method to use
    smooth_value: For 'floor' smoothing, the floor to use
    force: Ignore data that looks already tokenized
    lowercase: Lowercase the data
    tokenize: The tokenizer to use
Returns:
    'score': BLEU score,
    'counts': Counts,
    'totals': Totals,
    'precisions': Precisions,
    'bp': Brevity penalty,
    'sys_len': predictions length,
    'ref_len': reference length,
Examples:

    >>> predictions = ["hello there general kenobi", "foo bar foobar"]
    >>> references = [["hello there gen

हम स्कोर की गणना करने के लिए पूर्वानुमानों और लेबलों की तुलना करने के लिए `गणना` पद्धति का उपयोग करते हैं। पूर्वानुमानों और लेबलों दोनों को एक सूची बनाने की आवश्यकता है। विशिष्ट प्रारूप नीचे दिए गए उदाहरण में दिखाया गया है।

In [9]:
fake_preds = ["hello there", "general kenobi"]
fake_labels = [["hello there"], ["general kenobi"]]
metric.compute(predictions=fake_preds, references=fake_labels)

{'score': 0.0,
 'counts': [4, 2, 0, 0],
 'totals': [4, 2, 0, 0],
 'precisions': [100.0, 100.0, 0.0, 0.0],
 'bp': 1.0,
 'sys_len': 4,
 'ref_len': 4}

## डेटा प्रीप्रोसेसिंग

मॉडल में डेटा फीड करने से पहले, हमें डेटा को प्रीप्रोसेस करना होगा। प्रीप्रोसेसिंग टूल को टोकनाइज़र कहा जाता है। टोकनाइज़र पहले इनपुट को टोकनाइज़ करता है, फिर टोकन को प्री-मॉडल में आवश्यक संबंधित टोकन आईडी में परिवर्तित करता है। मॉडल के लिए आवश्यक इनपुट प्रारूप.

डेटा प्रीप्रोसेसिंग के उद्देश्य को प्राप्त करने के लिए, हम अपने टोकननाइज़र को इंस्टेंट करने के लिए AutoTokenizer.from_pretrained विधि का उपयोग करते हैं, जो सुनिश्चित करता है:

- हमें एक टोकननाइज़र मिलता है जो इससे मेल खाता हैएक-एक करके पूर्व-प्रशिक्षित मॉडल।
- निर्दिष्ट मॉडल चेकपॉइंट के अनुरूप टोकननाइज़र का उपयोग करते समय, हम मॉडल के लिए आवश्यक शब्दावली, अधिक सटीक रूप से, टोकन शब्दावली भी डाउनलोड करते हैं।

इस डाउनलोड की गई टोकन शब्दावली को कैश कर दिया जाएगा ताकि दोबारा उपयोग करने पर इसे दोबारा डाउनलोड न किया जा सके।

In [10]:
from transformers import AutoTokenizer
# `सेंटेंसपीस` इंस्टॉल करने की आवश्यकता: पिप इंस्टॉल सेंटेंसपीस
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading: 100%|██████████| 1.13k/1.13k [00:00<00:00, 466kB/s]
Downloading: 100%|██████████| 789k/789k [00:00<00:00, 882kB/s]
Downloading: 100%|██████████| 817k/817k [00:00<00:00, 902kB/s]
Downloading: 100%|██████████| 1.39M/1.39M [00:01<00:00, 1.24MB/s]
Downloading: 100%|██████████| 42.0/42.0 [00:00<00:00, 14.6kB/s]


उदाहरण के तौर पर हम जिस mBART मॉडल का उपयोग करते हैं, उसे लेते हुए, हमें स्रोत भाषा और लक्ष्य भाषा को सही ढंग से सेट करने की आवश्यकता है। यदि आप अन्य द्विभाषी कॉर्पोरा का अनुवाद करना चाहते हैं, तो कृपया [यहां] (https://huggingface.co/facebook/mbart-large) जांचें। -cc25) हम स्रोत और लक्ष्य भाषाओं की सेटिंग्स की जांच कर सकते हैं:

In [11]:
if "mbart" in model_checkpoint:
    tokenizer.src_lang = "en-XX"
    tokenizer.tgt_lang = "ro-RO"

टोकननाइज़र एकल टेक्स्ट या टेक्स्ट की एक जोड़ी को प्रीप्रोसेस कर सकता है। टोकननाइज़र प्रीप्रोसेसिंग के बाद प्राप्त डेटा पूर्व-प्रशिक्षित मॉडल के इनपुट प्रारूप से मिलता है।

In [12]:
tokenizer("Hello, this one sentence!")

{'input_ids': [125, 778, 3, 63, 141, 9191, 23, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

आप ऊपर जो टोकन आईडी (यानी इनपुट_आईडी) देख रहे हैं, वे आमतौर पर पूर्व-प्रशिक्षित मॉडल के नाम के साथ भिन्न होती हैं। इसका कारण यह है कि अलग-अलग पूर्व-प्रशिक्षित मॉडल पूर्व-प्रशिक्षण के दौरान अलग-अलग नियम निर्धारित करते हैं मॉडल समान हैं, टोकननाइज़र प्रीप्रोसेसिंग का इनपुट प्रारूप मॉडल आवश्यकताओं को पूरा करेगा। प्रीप्रोसेसिंग के बारे में अधिक जानकारी के लिए, कृपया [यह ट्यूटोरियल](https://huggingface.co/transformers/preprocessing.html) देखें।

टोकनिंग के अलावाएक वाक्य, हम वाक्यों की एक सूची को टोकनाइज़ भी कर सकते हैं।

In [13]:
tokenizer(["Hello, this one sentence!", "This is another sentence."])

{'input_ids': [[125, 778, 3, 63, 141, 9191, 23, 0], [187, 32, 716, 9191, 2, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}

नोट: मॉडल के लिए अनुवाद लक्ष्य तैयार करने के लिए, हम लक्ष्य के अनुरूप विशेष टोकन को नियंत्रित करने के लिए `as_target_tokenizer` का उपयोग करते हैं:

In [14]:
with tokenizer.as_target_tokenizer():
    print(tokenizer("Hello, this one sentence!"))
    model_input = tokenizer("Hello, this one sentence!")
    tokens = tokenizer.convert_ids_to_tokens(model_input['input_ids'])
# प्रिंट करें और विशेष टोकन देखें
    print('tokens: {}'.format(tokens))

{'input_ids': [10334, 1204, 3, 15, 8915, 27, 452, 59, 29579, 581, 23, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
tokens: ['▁Hel', 'lo', ',', '▁', 'this', '▁o', 'ne', '▁se', 'nten', 'ce', '!', '</s>']


यदि आप T5 पूर्व-प्रशिक्षित मॉडल के चेकपॉइंट्स का उपयोग कर रहे हैं, तो आपको विशेष उपसर्गों की जांच करने की आवश्यकता है। T5 मॉडल को किए जाने वाले विशिष्ट कार्यों को बताने के लिए विशेष उपसर्गों का उपयोग करता है। विशिष्ट उपसर्गों के उदाहरण इस प्रकार हैं:

In [15]:
if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "translate English to Romanian: "
else:
    prefix = ""

अब हम अपने प्रीप्रोसेसिंग फ़ंक्शन को बनाने के लिए सब कुछ एक साथ रख सकते हैं, जब हम नमूना को प्रीप्रोसेस करते हैं, तो हम यह सुनिश्चित करने के लिए पैरामीटर `truncation=True` का भी उपयोग करेंगे कि हमारे अत्यधिक लंबे वाक्यों को छोटा कर दिया जाए। डिफ़ॉल्ट रूप से, हम छोटे वाक्यों के लिए स्वचालित रूप से पैड लगाते हैं।

In [16]:
max_input_length = 128
max_target_length = 128
source_lang = "en"
target_lang = "ro"

def preprocess_function(examples):
    inputs = [prefix + ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

# लक्ष्य के लिए टोकननाइज़र सेटअप करें
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

उपरोक्त प्रीप्रोसेसिंग फ़ंक्शन एक नमूने या एकाधिक नमूना उदाहरणों को संसाधित कर सकता है। यदि यह एकाधिक नमूनों को संसाधित करता है, तो यह एकाधिक नमूनों के प्रीप्रोसेसिंग के परिणामों की एक सूची लौटाता है।

In [17]:
preprocess_function(raw_datasets['train'][:2])

{'input_ids': [[393, 4462, 14, 1137, 53, 216, 28636, 0], [24385, 14, 28636, 14, 4646, 4622, 53, 216, 28636, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[42140, 494, 1750, 53, 8, 59, 903, 3543, 9, 15202, 0], [36199, 6612, 9, 15202, 122, 568, 35788, 21549, 53, 8, 59, 903, 3543, 9, 15202, 0]]}

इसके बाद, डेटासेट डेटासेट में सभी नमूनों को प्रीप्रोसेसिंग फ़ंक्शन तैयार_ट्रेन_फीचर्स को सभी नमूनों पर लागू करने के लिए मैप फ़ंक्शन का उपयोग करके प्रीप्रोसेस किया जाता है।

In [18]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

100%|██████████| 2/2 [00:00<00:00,  3.89ba/s]


इससे भी बेहतर, अगली बार संसाधित होने पर पुनर्गणना से बचने के लिए लौटाए गए परिणाम स्वचालित रूप से कैश किए जाते हैं (लेकिन ध्यान रखें कि यदि इनपुट बदलता है, तो यह कैश से प्रभावित हो सकता है!) डेटासेट लाइब्रेरी फ़ंक्शन यह निर्धारित करने के लिए इनपुट पैरामीटर का पता लगाएगा यदि कोई परिवर्तन नहीं हैं, तो कैश्ड डेटा का उपयोग किया जाएगा। यदि परिवर्तन हैं, तो डेटा को पुन: संसाधित किया जाएगा। हालांकि, यदि इनपुट पैरामीटर नहीं बदलते हैं, तो जब आप चाहें तो कैश को साफ़ करना सबसे अच्छा है चांगई इनपुट। इसे साफ़ करने का तरीका `load_from_cache_file=False` पैरामीटर का उपयोग करना है। इसके अलावा, ऊपर इस्तेमाल किया गया `batched=True` पैरामीटर टोकननाइज़र की एक विशेषता है, क्योंकि यह इनपुट को संसाधित करने के लिए कई थ्रेड्स का उपयोग करेगा। समानांतर।

## ट्रांसफार्मर मॉडल को फाइन-ट्यून करें

अब जब डेटा तैयार है, तो हमें अपने पूर्व-प्रशिक्षित मॉडल को डाउनलोड और लोड करना होगा, और फिर पूर्व-प्रशिक्षित मॉडल को फाइन-ट्यून करना होगा क्योंकि हम seq2seq कार्य कर रहे हैं, हमें एक मॉडल वर्ग की आवश्यकता है जो इस कार्य को हल कर सके वर्ग `AutoModelForSeq2SeqLM` टोकननाइज़र के समान, `from_pretrained` विधि भी हमें मॉडल को डाउनलोड करने और लोड करने में मदद कर सकती है, और यह मॉडल को कैश भी करेगी ताकि हम मॉडल को बार-बार डाउनलोड न करें।

In [19]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

Downloading: 100%|██████████| 301M/301M [00:19<00:00, 15.1MB/s]


चूंकि हमारा फाइन-ट्यूनिंग कार्य मशीन अनुवाद है, और हम एक पूर्व-प्रशिक्षित seq2seq मॉडल लोड करते हैं, ऐसा कोई संकेत नहीं होगा कि मॉडल लोड करते समय कुछ बेमेल तंत्रिका नेटवर्क पैरामीटर हटा दिए जाएं (उदाहरण के लिए, पूर्व-प्रशिक्षित तंत्रिका नेटवर्क प्रमुख- प्रशिक्षित भाषा मॉडल को फेंक दिया जाता है, और मशीन अनुवाद के तंत्रिका नेटवर्क प्रमुख को यादृच्छिक रूप से प्रारंभ किया जाता है)।

`Seq2SeqTrainer` प्रशिक्षण उपकरण प्राप्त करने के लिए, हमें 3 और तत्वों की आवश्यकता है, जिनमें से सबसे महत्वपूर्ण है प्रशिक्षण सेटिंग्स/पैरामीटर [`Seq2SeqTrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html# ट्रांसफार्मर.Seq2SeqTrainingArguments) इस प्रशिक्षण सेटिंग में वे सभी गुण शामिल हैं जो प्रशिक्षण प्रक्रिया को परिभाषित कर सकते हैं

In [20]:
batch_size = 16
args = Seq2SeqTrainingArguments(
    "test-translation",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=False,
)

उपरोक्त मूल्यांकन_रणनीति = "युग" पैरामीटर प्रशिक्षण कोड को बताता है कि हम प्रति युग एक बार सत्यापन मूल्यांकन करेंगे।

इस नोटबुक से पहले बैच_आकार को ऊपर परिभाषित किया गया है।

चूँकि हमारा डेटासेट बड़ा है और `Seq2SeqTrainer` मॉडलों को सहेजता रहेगा, हमें इसे अधिकतम `save_total_limit=3` मॉडलों को सहेजने के लिए कहना होगा।

अंत में, हमें अपने संसाधित इनपुट को मॉडल में फीड करने के लिए एक डेटा कोलेटर की आवश्यकता होती है।

In [21]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

Seq2SeqTrainer को स्थापित करने के लिए आखिरी चीज मूल्यांकन पद्धति को परिभाषित करना है। हम ऐसा करने के लिए मीट्रिक का उपयोग करते हैं। हम मूल्यांकन के लिए मॉडल पूर्वानुमान भेजने से पहले कुछ पोस्ट-प्रोसेसिंग भी करेंगे:

In [22]:
import numpy as np

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

# लेबल में -100 बदलें क्योंकि हम उन्हें डिकोड नहीं कर सकते।
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

# कुछ सरल पोस्ट-प्रोसेसिंग
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

अंत में, सभी पैरामीटर/डेटा/मॉडल को `Seq2SeqTrainer` पर पास करें

In [23]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

फाइन-ट्यूनिंग प्रशिक्षण करने के लिए `ट्रेन` विधि को कॉल करें।

In [ ]:
trainer.train()

अंत में, यह जांचना न भूलें कि किसी मॉडल को कैसे अपलोड किया जाए और उसे [🤗 मॉडल हब](https://huggingface.co/models) पर अपलोड किया जाए, फिर आप बस मॉडल नाम का उपयोग करके अपने मॉडल का उपयोग कर सकते हैं इस नोटबुक की शुरुआत में.